## Sample Data Generation

In [8]:
from samples import generate_sample_data
import json

sample_data = generate_sample_data(1000, 1000, 200)
with open("samples/data.json", "w") as f:
    json.dump(sample_data, f)

In [1]:
import json
from random import randrange
with open("samples/data.json", "r") as f:
    sample_data = json.load(f)

user_ratings = sample_data["ratings"]
test_set_size = int(len(user_ratings) * len(user_ratings[0]) * 0.2)
test_set = [(randrange(0, len(user_ratings)), randrange(0, len(user_ratings[0]))) for _ in range(test_set_size)]

In [7]:
user_ratings = [
    [5, 4, 5, 3, 3],
    [3, 2, 2, 4, 1],
    [3, 4, 3, 5, 4],
    [5, 1, 4, 2, 4],
    [2, 3, 4, 1, 1],
    [2, 3, 4, 2, 5],
]

test_set = [
    (0, 0),
    (0, 3),
    (1, 1),
    (1, 4),
    (2, 0),
    (2, 4),
    (3, 2),
    (4, 1),
    (4, 3),
    (5, 0),
]

In [4]:
user_ratings = [
    [5, 4, 4, -1, 5],
    [-1, 3, 5, 3, 4],
    [5, 2, -1, 2, 3],
    [-1, 2, 3, 1, 2],
    [4, -1, 5, 4, 5],
    [5, 3, -1, 3, 5],
    [3, 2, 3, 2, -1],
    [5, 3, 4, -1, 5],
    [4, 2, 5, 4, -1],
    [5, -1, 5, 3, 4]
]
test_set = [(0, 4), (1, 3), (2, 3), (3, 1), (4, 2),
                   (5, 0), (6, 1), (7, 1), (8, 0), (9, 0)]

In [ ]:
from predictors.least_squares import LeastSquaresPredictor
from utils import get_test_set_matrix, remove_test_set, root_mean_square_error


training_data = remove_test_set(user_ratings, test_set)
test_data = get_test_set_matrix(user_ratings, test_set)
baseline = LeastSquaresPredictor(training_data=training_data, lmda=0.25)
baseline.train()
test_predictions = baseline.predict(test_set)
all_predictions = baseline.predict_all()
print(f"{test_predictions = }")
print(f"{all_predictions = }")
rmse_training = root_mean_square_error(all_predictions, training_data)
rmse_test = root_mean_square_error(all_predictions, test_data)
print(f"{rmse_training = }")
print(f"{rmse_test = }")

Constructing relevant matrices...
Calculating user and item biases...


/Users/matthewngan/ESTR3302-project/predictors/least_squares.py:81: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  self.b = spsolve(AAT, rhs)


Training done.
Predicting entries...


100%|██████████| 200000/200000 [00:00<00:00, 2456825.04it/s]


Finish predicting entries.
Predicting all...


100%|██████████| 1000/1000 [00:00<00:00, 2530.91it/s]

Finish predicting all.
test_predictions = array([3.09117081, 3.10550883, 3.12030311, ..., 3.11078917, 3.1101873 ,
       3.08228738], shape=(200000,))
all_predictions = array([[3.09294839, 3.12755487, 3.11496811, ..., 3.12567891, 3.11557652,
        3.06371067],
       [3.07693211, 3.11153859, 3.09895183, ..., 3.10966263, 3.09956024,
        3.04769438],
       [3.09329141, 3.12789788, 3.11531113, ..., 3.12602192, 3.11591954,
        3.06405368],
       ...,
       [3.07644045, 3.11104692, 3.09846017, ..., 3.10917096, 3.09906858,
        3.04720272],
       [3.08974041, 3.12434689, 3.11176013, ..., 3.12247093, 3.11236854,
        3.06050269],
       [3.10323256, 3.13783904, 3.12525228, ..., 3.13596308, 3.12586069,
        3.07399483]], shape=(1000, 1000))


In [ ]:
import numpy as np
from predictors.neighbor_correlations import Correlation, NeighborCorrelationsPredictor
from utils.neighbor_selection import most_similar, two_most_similar_skip_masked

# baseline.predict_all = lambda : np.array(
#     [
#         [np.nan, 2.7, 3.3, np.nan, 4.5],
#         [4.1, np.nan, 3.5, 4.9, np.nan],
#         [np.nan, 3.8, 2.5, 4.2, np.nan],
#         [2.8, 3.1, np.nan, 2.6, 4.8],
#         [3.3, np.nan, 3.7, np.nan, 2.4],
#         [np.nan, 3.9, 4.0, 1.5, 3.9],
#     ]
# )

improved = NeighborCorrelationsPredictor(baseline=baseline, correlation=Correlation.ITEM)
improved.train(most_similar)
test_predictions = improved.predict(test_set)
all_predictions = improved.predict_all()
print(f"{test_predictions = }")
print(f"{all_predictions = }")

Calculating cosine similarity coefficients...
Predicting all...


100%|██████████| 1000/1000 [00:00<00:00, 2899.24it/s]


Finish predicting all.
Making neighbor table...


100%|██████████| 1000/1000 [00:05<00:00, 167.60it/s]


Finish training.
Predicting entries...


100%|██████████| 200000/200000 [00:00<00:00, 2571394.24it/s]


Finish predicting entries.
Predicting all...


100%|██████████| 1000/1000 [00:00<00:00, 2978.51it/s]


Finish predicting all.


In [5]:
import numpy as np
from predictors.latent_factor import LatentFactorPredictor

sample_user_ratings = [
    [3, 4, 5, 3, 2, 3],
    [3, 2, 3, 4, 2, 1],
    [4, 4, 4, 5, 3, 2],
    [3, 5, 4, 4, 3, 4],
    [2, 1, 2, 2, 3, 1],
    [3, 5, 5, 4, 4, 3],
    [3, 5, 5, 3, 2, 2],
    [2, 3, 3, 2, 1, 2],
]
sample_test_set = [
    (0, 0),
    (1, 1),
    (2, 3),
    (2, 4),
    (3, 0),
    (3, 1),
    (5, 1),
    (5, 4),
    (6, 0),
    (6, 2),
    (7, 1),
    (7, 3),
]
training_data = remove_test_set(sample_user_ratings, sample_test_set)
test_data = get_test_set_matrix(sample_user_ratings, sample_test_set)
u, i = training_data.shape
k = 2
latent = LatentFactorPredictor(
    training_data=training_data,
    k=k,
    p=np.full((k, u), 1, dtype=np.float64),
    q=np.full((k, i), 1, dtype=np.float64),
    lmda=0.2,
)
latent.train(20)
all_predictions = latent.predict_all()
test_predictions = latent.predict(sample_test_set)
print(f"{all_predictions = }")
print(f"{latent.p = }")
print(f"{latent.q = }")

all_predictions = array([[2.73402602, 4.48879132, 4.30697551, 3.2186042 , 1.96912014,
        2.81803155],
       [2.83475932, 2.56022117, 2.90042499, 3.28621128, 2.69179878,
        1.50348167],
       [3.74631254, 3.7619743 , 4.11600953, 4.3521493 , 3.43987108,
        2.24331636],
       [3.38052349, 4.72745277, 4.71039242, 3.9596552 , 2.69019862,
        2.9270728 ],
       [2.17435686, 1.27615303, 1.71072411, 2.50389476, 2.27819342,
        0.68744758],
       [3.24193673, 4.89639618, 4.78844221, 3.80615816, 2.46728672,
        3.05278956],
       [2.65612574, 4.34602767, 4.1731457 , 3.12653497, 1.9176295 ,
        2.7276686 ],
       [1.77062656, 3.13429637, 2.95917071, 2.08998444, 1.20470111,
        1.97895441]])
latent.p = array([[ 0.88963584,  1.50009581,  1.87805526,  1.32698965,  1.34032657,
         1.17251361,  0.86838852,  0.51346081],
       [ 1.58503227,  0.41031614,  0.76514332,  1.47530906, -0.09021009,
         1.62844766,  1.53111621,  1.16032704]])
latent.q = arra

In [5]:
rmse_training = root_mean_square_error(all_predictions, training_data)
rmse_test = root_mean_square_error(all_predictions, test_data)
print(f"{rmse_training = }")
print(f"{rmse_test = }")

Calculating mean-square error


100%|██████████| 1000/1000 [00:08<00:00, 119.93it/s]


Calculating mean-square error


100%|██████████| 1000/1000 [00:06<00:00, 166.47it/s]

rmse_training = np.float64(0.5998593466592971)
rmse_test = np.float64(0.6292351490204294)
